In [ ]:
## if itk is not installed
!pip install itk

In [103]:
import itk
import numpy as np
import os

In [13]:
os.getcwd()

'C:\\Users\\chunr\\OneDrive - University of Iowa\\SPIROMICS_DICOM2Nifti'

In [104]:
PixelType = itk.ctype("short")
image_t = itk.Image[PixelType,3]


In [105]:
dicomFN.SetUseSeriesDetails(True) # Use more information to identify individual 3D image
dicomFN.SetDirectory("./SampleDicom/unrarred/")

In [ ]:
## Different method to access tag value
## -------------------------------------
#dicomIO.SetMetaDataDictionary()
#metad['0008|0000']
#labelId=""
#dicomIO.GetLabelFromTag('0008|0000',labelId)
#dicomIO.GetValueFromTag('0008|0000',labelId)

In [108]:
lab2Keep = ('0008|0000','0008|0008','0008|0012','0008|0013','0008|0020','0008|0021',
'0008|0022','0008|0023','0008|0030','0008|0031','0008|0032','0008|0033',
'0008|0060','0008|1030','0008|103e','0010|0000','0010|0010','0010|0020',
'0018|0000','0018|0050','0020|0000','0020|000d','0020|000e','0028|0000',
'0028|0010','0028|0011','0028|0030','0028|0101')
#GenericGroupLength,ImageType, InstanceCreationDate, InstanceCreationTime,
#StudyDate, SeriesDate,AcquisionDate,ContentDate,StudyTime,SeriesTime,
#AcquisionTime,ContentTime,Modality,StudyDescription,SeriesDescription,
#GenericGroupLength,PatientName, PatientID,GenericGroupLength,SliceThickness,
#GenericGroupLength,StudyInstanceUID,SeriesInstanceUID,GenericGroupLength,
#Rows,Columns,PixelSpacing,BitsStored

for label in metad.GetKeys():
    if label in lab2Keep:
        continue
    else:
        metad.Erase(label)
        
dicomIO.SetMetaDataDictionary(metad)        
nameGenerator = itk.GDCMSeriesFileNames.New()
nameGenerator.SetUseSeriesDetails(True)  ## in case there are multiple images in the same series folder
nameGenerator.SetMetaDataDictionary(metad)
nameGenerator.SetInputDirectory("./test2") ## Using this method, one can use nameGenerator.GetInputFilenames()
nameGenerator.GetOutputFileNames()

## The code below is not finished and incorrect. The DICOM files could be not be processed as a series using ImageSeriesWriter.
## I am not sure if the ImageSeriesReader should be used. Instead, writing DICOM series should use ImageFileReader and SetImageIO.
## Refer to "Reading a 2D DICOM Series and Writing a 2D DICOM Series" in ITKUserGuide. It reads a 3D DICOM image but writes 2D 
## series which I don't quites get it. 

reader = itk.ImageSeriesReader[image_t].New()
dicomIO = itk.GDCMImageIO.New()
dicomFN = itk.GDCMSeriesFileNames.New()
reader.SetImageIO(dicomIO)

uids = dicomFN.GetSeriesUIDs()
fnames = dicomFN.GetFileNames(uids[0])
reader.SetFileNames(fnames)
reader.Update()
image = reader.GetOutput()
metad = dicomIO.GetMetaDataDictionary()

image_2d = itk.Image[PixelType,2]
dcmWriter = itk.ImageFileWriter[image_2d,image_2d].New() ## ImageSeriesWriter does not work for 3D dicom series writing
dicom_io = itk.GDCMImageIO.New()
dcmWriter.SetImageIO(dicom_io)




In [ ]:
## Write DICOM to nifti format
## -------------------------------------
#nifWriter = itk.ImageFileWriter[image_t].New()
#nifti_io = itk.NiftiImageIO.New()
#nifWriter.SetFileName("test.nii.gz")
#nifWriter.SetInput(image)
#nifWriter.SetImageIO(nifti_io)
#nifWriter.Update()